# per ogni input controllare che l'ufficio sia presente nella lista dei codici uffici

#### Inseriamo ufficio ( tramite id )
#### Inseriamo ente ( tramite id ) numero, anno , ricerca_iqera 
#### per ora entriamo solo nella pagina non facciamo lo scraper dei risultati

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
def scraper(ufficio,ente,anno,numero,natura):
    """
    ufficio è l'id che si trova nel file uffici_finanziari (in code)
    ente è l'id da ricavare da enti.json   (esempio 001 )
    numero da file input (rg / decreto / etc )
    anno anno con 4 cifre da file input
    natura  DI  AV  /  EM  OR
    """


    session = requests.Session()
    initial_url = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
    session.get(initial_url) 

    url1 = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
    params1 = {
        'action': 'scegliufficio',
        'ufficio': ufficio,
    }
    headers = {
        'Connection': 'keep-alive',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept': '*/*',
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    body1 = {
        'ufficio': ufficio,
    }
    response1 = session.post(url1, headers=headers, params=params1, data=body1)
    
    if response1:
        url2 = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
        params2 = {
            'action': 'scegliente',
        }
        body2 = {
            'ente': ente,
            'anno': anno,
            'numero': numero,
            'natura' : natura,
            'snumero' : ""
        }
        response2 = session.post(url2, headers=headers, params=params2, data=body2)
        if response2 :
            soup2 = BeautifulSoup(response2.text, 'html.parser')
            if 'non trovato' in response2.text:
                rep = str(numero) + ' - ' + str(anno)
                return [ufficio, natura, ente, rep, '-', '-', '-','no']
            else :
                compila = 'no'
                tabella = soup2.find('div', class_='panel-body')

                if  tabella.find('span', string="Atto tassato dall'ufficio di: "): 
                    office = tabella.find('span', string="Atto tassato dall'ufficio di: ").find_next_sibling('span').get_text(strip=True)
                else :
                    office = '-'   

                if tabella.find('span', string="Tipologia atto: "):
                    atto = tabella.find('span', string="Tipologia atto: ").find_next_sibling('span').get_text(strip=True)
                else :
                    atto = '-'  

                if tabella.find('span', string="Emesso da: "):
                    emesso_da = tabella.find('span', string="Emesso da: ").find_next_sibling('span').get_text(strip=True)
                else :
                    emesso_da = '-'   

                if tabella.find('span', string="n. Repertorio - anno: "):
                    repertorio = tabella.find('span', string="n. Repertorio - anno: ").find_next_sibling('span').get_text(strip=True)
                else :
                    repertorio = '-'  

                if tabella.find('span', string="Attore: "):
                    attore = tabella.find('span', string="Attore: ").find_next_sibling('span').get_text(strip=True)
                else:
                    attore='-'

                if tabella.find('span', string="Convenuto: "):
                    convenuto = tabella.find('span', string="Convenuto: ").find_next_sibling('span').get_text(strip=True)
                else:
                    convenuto ='-'

                if tabella.find('span', string="Stato dell'atto: "):
                    stato = tabella.find('span', string="Stato dell'atto: ").find_next_sibling('span').get_text(strip=True)
                else:
                    stato= '-'

                if 'compilaf24.php' in response2.text:  # Check if the link is present in the whole body
                    compila = 'si'



                return [office,atto,emesso_da,repertorio,attore,convenuto,stato,compila]
        else :
            rep = str(numero) + ' - ' + str(anno)
            return [ufficio, natura, ente, rep, '-', '-', '-','no']

    else:
       return 'ufficio non trovato'   

#### cambia la natura dell'ente in base al tipo diu ricerca ( se monitoria o ppt)

In [4]:
list_natura_monitoria = ['DI','AV']
list_natura_ppt = ['EM','OR']

def tipo_scraper(contratto,debitore, tipo, ufficio, lista_enti, anno, numero,row_number):
    results = []
    try:
        if tipo == 'monitoria':
            natura_list = list_natura_monitoria
        elif tipo == 'ppt':
            natura_list = list_natura_ppt
        else:
            raise ValueError("Invalid tipo value")

        for natura in natura_list:
            for ente in lista_enti:
                result=''
                result = scraper(ufficio, ente, anno, numero, natura)
                if isinstance(result, list):
                    if len(result) == 8:
                        results.append({
                            "contratto":contratto,
                            "debitore":debitore,
                            'ufficio': result[0],
                            'atto': result[1],
                            'emesso da': result[2],
                            'repertorio': result[3],
                            'attore': result[4],
                            'convenuto': result[5],
                            'stato': result[6],
                            "compila":result[7]
                        })
                    else:
                        rep = str(numero) + ' - ' + str(anno)
                        results.append({
                            "contratto":contratto,
                            "debitore":debitore,
                            'ufficio': ufficio,
                            'natura': natura,
                            'ente': ente,
                            'repertorio': rep,
                            'attore': '-',
                            'convenuto': '-',
                            'stato': '-',
                            "compila":'no'
                        })
                else:
                    print(result)
    except Exception as e:
        print(f'Riga con errore: {row_number}')
        print(f"Error occurred: {e}")
    return results


#### sistemiamo l'input prima di mandare lo scaper 

In [5]:
from Pulizia import clean , split_and_expand
import pandas as pd
import json
from datetime import date
import numpy as np
today = date.today()


In [6]:
uffici = pd.read_excel('Resources/uffici_finanziari.xlsx')
uffici = uffici.iloc[1:]
uffici['office'] = uffici['office'].str.lower()
uffici_ut = uffici[uffici['office'].str.contains('ut')]

In [7]:
input = pd.read_excel('Input/Raw/perimetro_03_06.xlsx')

In [8]:
n_columns = input.filter(regex='^N')
contains_dash = n_columns.applymap(lambda x: '-' in str(x) and '/' not in str(x))
n_columns = n_columns.applymap(lambda x: x.replace('-', '/') if '-' in str(x) else x)
input[n_columns.columns] = n_columns

In [9]:
input = input[input['N. RG'].str.contains('/', na=True) | input['N. RG'].isna()]
input = input[input['N. Decreto'].str.contains('/', na=True) | input['N. Decreto'].isna()]
input = input[input['N.R.G.E PPT'].str.contains('/', na=True) | input['N.R.G.E PPT'].isna()]
input = input[input['N° Repertorio'].str.contains('/', na=True) | input['N° Repertorio'].isna()]
input = input[input['N° Rep PPT'].str.contains('/', na=True) | input['N° Rep PPT'].isna()]
input = input[input['N. Cronologico'].str.contains('/', na=True) | input['N. Cronologico'].isna()]

In [10]:
input = input.drop_duplicates()

In [11]:


input = clean(input)
#input['tipo_ricerca'] = input['RICERCA IQERA'].str.lower()
#input = input.drop(columns=['RICERCA IQERA'])

In [12]:


mapping = {'albenga':'savona',
    'chiavenna':'sondrio',
    'cossato':'biella',
    'macomer':'nuoro',
    'isili':'nuoro',
    'chiaromonte': 'lagonegro',
    'orzinuovi' : 'brescia',
    'ovada': 'acquiterme',
    'pavullo nel frignano':'modena',
    'ponte san pietro':'bergamo',
    'romano di lombardia':'bergamo',
    'san benedetto del tronto':'ascoli piceno',
    'teano':'caserta',
    'venezia 2 - mestre' : 'venezia',
    'venosa':'potenza',
    'vimercate':'monza',
    'acerra' : 'casoria',
    'torre annunziata':'napoli',
    'napoli nord in aversa':'napoli',
    'napoli nord':'napoli',
    'san giorgio ionico':'taranto',
    "rometta" :'barcellona pozzo di gotto',
    'taurianova':'palmi',
    'putignano':'gioia del colle',
    'dolo':'venezia',
    'ugento':'casarano',
    'grumello del monte':'bergamo',
    'todi':'perugia',
    'gragnano':'castellammare di stabia',
    'tribunale  di vicenza':'vicenza',
    'marigliano':'nola',
    "reggio nell'emilia": 'reggio emilia',
    'montoro':'avellino',
    'nocera inferiore':'pagani',
    "chieti - sezione distaccata di ortona":'chieti',
    'la maddalena':'olbia',
    'san giovanni in fiore':'cosenza',
    'poggio mirteto':'rieti',
    'volterra':'pontedera',
    'tribunale della spezia':'la spezia',
    'arcidosso':'grosseto',
    'martina franca':'taranto',
    'naso':'messina',  # forse
    'scalea':'paola',
    'cariati':'rossano',
    'montecorvino rovella':'salerno',
    "sant'anastasia":'napoli',
    'forlì':"forli'"
    }

input['Foro'] = (input['Foro']
                    .str.lower()
                    .replace(mapping, regex=True)
                    .str.strip()
                   )
# input
input = input.drop_duplicates()

In [13]:
def find_code_for_foro(df1, df2):
    # Initialize an empty list to store the results
    results = []

    # Iterate over each row in df2
    for index2, row2 in df2.iterrows():
        foro = row2['Foro']
        
        # Find matching rows in df1
        matching_rows = df1[df1['office'].str.contains(foro)]

        # Add matching codes to the results list
        for index1, row1 in matching_rows.iterrows():
            results.append({
                'code_office': row1['code'],
                'office': row1['office'],
                'Foro': row2['Foro']
            })

    # Convert results to a DataFrame
    result_df = pd.DataFrame(results)
    return result_df
result_df = find_code_for_foro(uffici_ut, input)
result_df = result_df.drop_duplicates()

In [14]:

input = pd.merge(input, result_df, on='Foro', how='left')
input = (input
        .drop_duplicates())
input['Tribunale/Gdp'] = (input['Tribunale/Gdp']
                          .str.lower()
                          .str.strip()
)

input = input.drop_duplicates()

In [15]:
input[input['code_office'].isna()]['Foro'].drop_duplicates()

Series([], Name: Foro, dtype: object)

#### sopra abbiamo aggiunto il codice dell'ufficio

#### ora dobbiamo aggiungere i codici degli enti

In [16]:
with open('Resources/enti_completi_ut.json', 'r') as file:
    enti_ut = json.load(file)

enti_decoded = {}

for key, values in enti_ut.items():
    tribunale = [item.split(" - ")[0] for item in values if "TRIB" in item.upper()]
    giudice = [item.split(" - ")[0] for item in values if "GIUD" in item.upper()]
    enti_decoded[key] = {"tribunale": tribunale, "giudice": giudice}
# enti_decoded    

In [17]:
def get_codes(row):
    office = row['office']
    type_ = row['Tribunale/Gdp']
    if office in enti_decoded and type_ in enti_decoded[office]:
        return enti_decoded[office][type_]
    else:
        return []

# Apply the function to each row and create a new column 'codes'
input['code_ente'] = input.apply(get_codes, axis=1)


In [18]:
# Convert the 'code_ente' lists to strings
input['code_ente'] = input['code_ente'].apply(lambda x: ','.join(x))


potential_value_vars = [
    'nr.decreto', 'year.decreto', 'nr.rg', 'year.rg',
    'nr.repertorio', 'year.repertorio', 'nr.crono', 'year.crono',
    'nr.rgeppt', 'year.rgeppt', 'nr.repppt', 'year.repppt'
]
value_vars = [col for col in potential_value_vars if col in input.columns]
# Melt the DataFrame to 'pivot longer'
df_long = pd.melt(input, 
                  id_vars=['NR_Contratto', 'Debitore','Tribunale/Gdp','Foro','Cliente','tipo_ricerca','code_office','office','code_ente'], 
                  value_vars=value_vars, 
                  var_name='type', 
                  value_name='value')

# Split the 'type' column into 'attribute' and 'category'
df_long[['attribute', 'category']] = df_long['type'].str.split('.', expand=True)

# Pivot the DataFrame to get the desired format
input = df_long.pivot_table(index=['NR_Contratto', 'Debitore','Tribunale/Gdp','Foro','Cliente','tipo_ricerca','code_office','office','code_ente','category'], 
                               columns='attribute', 
                               values='value', 
                               aggfunc='first').reset_index()

# Flatten the columns
input.columns.name = None
input = input.reset_index(drop=True)

# Convert the 'code_ente' column back into a list
input['code_ente'] = input['code_ente'].apply(lambda x: x.split(',') if isinstance(x, str) else x)


In [19]:
input = input.drop(columns={'category'})
input['code_ente'] = input['code_ente'].apply(tuple)
input = input.drop_duplicates()
input['code_ente'] = input['code_ente'].apply(list)

In [20]:
input.to_excel('Input/Cleaned/2024_06_03_input_v1.xlsx',index=False)

In [21]:
input

,NR_Contratto,Debitore,Tribunale/Gdp,Foro,Cliente,tipo_ricerca,code_office,office,code_ente,nr,year
0,AA1200001768_06112018,SGOBBA IVAN,giudice,brescia,A2A,monitoria,TM4,brescia ut dpbs,"[003, 005, 006, 009]",4693,2020
1,AA1200001768_06112018,SGOBBA IVAN,giudice,brescia,A2A,monitoria,TM4,brescia ut dpbs,"[003, 005, 006, 009]",1236,2020
3,AA1200001768_06112018,SGOBBA IVAN,giudice,brescia,A2A,monitoria,TMJ,brescia 1 ut dpbs,"[003, 005, 006, 009]",4693,2020
4,AA1200001768_06112018,SGOBBA IVAN,giudice,brescia,A2A,monitoria,TMJ,brescia 1 ut dpbs,"[003, 005, 006, 009]",1236,2020
6,AA1200002409_06112018,TAVARES FARIAS CARLOS ERIK,giudice,brescia,A2A,monitoria,TM4,brescia ut dpbs,"[003, 005, 006, 009]",4609,2020
...,...,...,...,...,...,...,...,...,...,...,...
5812,XUB385075,BUCCIONE PIERO,tribunale,chieti,Sirio,monitoria,TAH,chieti ut dpch,[001],1001,2023
5814,XUB385803,STOP CAFFE' SNC DI GENNARELLI GENNARO E GUARAG...,tribunale,civitavecchia,Sirio,monitoria,TJ3,civitavecchia ut dprm2,[001],193,2023
5816,XUB385803,STOP CAFFE' SNC DI GENNARELLI GENNARO E GUARAG...,tribunale,civitavecchia,Sirio,monitoria,TJF,civitavecchia - ante 01.02.2015 ut dprm1,"[001, 004, 005, 006, 007, 011]",193,2023
5818,XUB386298,PESCE MAURO,tribunale,pescara,Sirio,monitoria,TAU,pescara ut dppe,"[001, 005, 006]",4962,2021


In [39]:
# new_df = input[2000:2500]

new_df = input[800:1000]

results =( new_df 
    .apply(lambda row: tipo_scraper(contratto=row['NR_Contratto'],
                                    debitore=row['Debitore'],
                                    tipo= row['tipo_ricerca'],
                                    ufficio= row['code_office'],
                                    lista_enti=row['code_ente'], 
                                    anno=row['year'],
                                    numero= row['nr'],
                                    row_number=row.name
                                    ),
                                    axis=1)
)

Riga con errore: 1371
Error occurred: 'NoneType' object has no attribute 'find'
Riga con errore: 1468
Error occurred: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


In [47]:
# 1371 , 1468 
input.loc[1372]

NR_Contratto                                      CRCO13559620.074
Debitore                                          DE VANNA GAETANA
Tribunale/Gdp                                            tribunale
Foro                                                         trani
Cliente                                                   MBCREDIT
tipo_ricerca                                                   ppt
code_office                                                    TUZ
office                                              trani  ut dpbt
code_ente        [004, 008, 009, 010, 011, 012, 013, 014, 015, ...
nr                                                             692
year                                                          2021
Name: 1372, dtype: object

Risultati in un df

In [40]:
filtered_data = [item for item in results if item != []]
flattened_results = [item for sublist in filtered_data for item in (sublist if isinstance(sublist, list) else [sublist])]
df = pd.DataFrame(flattened_results)
df = df.drop_duplicates()
df

,contratto,debitore,ufficio,atto,emesso da,repertorio,attore,convenuto,stato,compila
0,CRCO12560356.074,GALULLO GIOVANNI,TNL,EM,001,3366 - 2022,-,-,-,no
1,CRCO12560356.074,GALULLO GIOVANNI,TNL,EM,002,3366 - 2022,-,-,-,no
2,CRCO12560356.074,GALULLO GIOVANNI,MILANO 1 UT DPMI1,ESECUZIONE MOBILIARE n. 3366 del 2022,TRIB.CIV.SEZ. ESECUZIONI,3981 - 2022,MBCREDIT SOLUTIONS S.P.A.,GALULLO,ATTO IN FASE DI TASSAZIONE,no
3,CRCO12560356.074,GALULLO GIOVANNI,TNL,EM,010,3366 - 2022,-,-,-,no
4,CRCO12560356.074,GALULLO GIOVANNI,TNL,EM,011,3366 - 2022,-,-,-,no
...,...,...,...,...,...,...,...,...,...,...
2442,CRS5239028,DE SIENA VINCENZO,TER,AV,025,13987 - 2022,-,-,-,no
2443,CRS5239028,DE SIENA VINCENZO,TER,AV,027,13987 - 2022,-,-,-,no
2444,CRS5239028,DE SIENA VINCENZO,TER,AV,028,13987 - 2022,-,-,-,no
2445,CRS5239028,DE SIENA VINCENZO,TER,AV,039,13987 - 2022,-,-,-,no


In [41]:
cliente = input[['NR_Contratto','Cliente']]
df = pd.merge(df,cliente,how='left', left_on='contratto',right_on='NR_Contratto' )
df.drop(columns=['NR_Contratto'], inplace=True)
df = df.drop_duplicates()
df

,contratto,debitore,ufficio,atto,emesso da,repertorio,attore,convenuto,stato,compila,Cliente
0,CRCO12560356.074,GALULLO GIOVANNI,TNL,EM,001,3366 - 2022,-,-,-,no,MBCREDIT
1,CRCO12560356.074,GALULLO GIOVANNI,TNL,EM,002,3366 - 2022,-,-,-,no,MBCREDIT
2,CRCO12560356.074,GALULLO GIOVANNI,MILANO 1 UT DPMI1,ESECUZIONE MOBILIARE n. 3366 del 2022,TRIB.CIV.SEZ. ESECUZIONI,3981 - 2022,MBCREDIT SOLUTIONS S.P.A.,GALULLO,ATTO IN FASE DI TASSAZIONE,no,MBCREDIT
3,CRCO12560356.074,GALULLO GIOVANNI,TNL,EM,010,3366 - 2022,-,-,-,no,MBCREDIT
4,CRCO12560356.074,GALULLO GIOVANNI,TNL,EM,011,3366 - 2022,-,-,-,no,MBCREDIT
...,...,...,...,...,...,...,...,...,...,...,...
8724,CRS5239028,DE SIENA VINCENZO,TER,AV,025,13987 - 2022,-,-,-,no,MBCREDIT
8733,CRS5239028,DE SIENA VINCENZO,TER,AV,027,13987 - 2022,-,-,-,no,MBCREDIT
8742,CRS5239028,DE SIENA VINCENZO,TER,AV,028,13987 - 2022,-,-,-,no,MBCREDIT
8751,CRS5239028,DE SIENA VINCENZO,TER,AV,039,13987 - 2022,-,-,-,no,MBCREDIT


Righe con risultati

In [42]:
df[(df['attore'] != '-') | (df['convenuto'] != '-') ]

,contratto,debitore,ufficio,atto,emesso da,repertorio,attore,convenuto,stato,compila,Cliente
2,CRCO12560356.074,GALULLO GIOVANNI,MILANO 1 UT DPMI1,ESECUZIONE MOBILIARE n. 3366 del 2022,TRIB.CIV.SEZ. ESECUZIONI,3981 - 2022,MBCREDIT SOLUTIONS S.P.A.,GALULLO,ATTO IN FASE DI TASSAZIONE,no,MBCREDIT
60,CRCO12832445.074,GATTO GIUSEPPE,MESSINA UT DPME,DECRETO INGIUNTIVO n. 99 del 2021,TRIBUNALE DI MESSINA,2464/21 - 2021,MARTE SPV S.R.L.,-,ATTO REGISTRATO,no,MBCREDIT
74,CRCO12930893.074,MISTRETTA MICHELE,PALERMO 2 UT DPPA,DECRETO INGIUNTIVO n. 404 del 2023,GIUDICE DI PACE DI PALERMO,1450 - 2023,IFIS NPL INVESTING S.P.A.,-,ATTO REGISTRATO,no,MBCREDIT
112,CRCO13016181.074,ACQUAVIVA GIROLAMO,MILANO 1 UT DPMI1,ESECUZIONE MOBILIARE n. 4719 del 2021,TRIB.CIV.SEZ. ESECUZIONI,1022 - 2021,BOSCOGRANDE S.N.C. DI PALLADIU,EL GAMAL,ATTO REGISTRATO,no,MBCREDIT
124,CRCO13099181.074,DEL ROSARIO IVONNE,LANCIANO UT DPCH,ESECUZIONE MOBILIARE n. 351 del 2021,TRIBUNALE DI LANCIANO,351 - 2021,COMPASS BANCA SPA,MEMMO LUCIA GIUSEPPINA,ATTO REGISTRATO,no,MBCREDIT
422,CRCO13420190.074,MUGHAL ABDUL KHALIQ,BUSTO ARSIZIO UT DPVA,ESECUZIONE MOBILIARE n. 1860 del 2022,TRIBUNALE,9388/23 - 2023,MBCREDIT SOLUTIONS S.P.A.,MUGHAL,ATTO REGISTRATO,no,MBCREDIT
541,CRCO13559620.074,DE VANNA GAETANA,TRANI UT DPBT,ESECUZIONE MOBILIARE n. 692 del 2021,TRIB.TRANI-ESEC.MOB-IMMOB INF.,692/21 - 2021,ITACAPITAL SRL,PAPA,ATTO REGISTRATO,no,MBCREDIT
559,CRCO13594450.074,D'ALESSIO CAMMILLA,LUCCA UT DPLU,DECRETO INGIUNTIVO n. 839 del 2022,TRIBUNALE,1272 - 2022,AURELIA SPV S.R.L.,ANTICO CAFFE' DI LUCCA SRL,ATTO REGISTRATO,no,MBCREDIT
573,CRCO13675340.074,SCHEMBRI CALOGERO,RAVENNA UT DPRA,DECRETO INGIUNTIVO n. 749 del 2022,TRIBUNALE DI RAVENNA,1183/23 - 2023,KRUK INVESTIMENTI S.R.L.,TONI NICOLETTA,ATTO REGISTRATO,no,MBCREDIT
751,CRCO14140402.074,SERRA GIUSEPPE,CASSINO UT DPFR,ESECUZIONE MOBILIARE n. 959 del 2022,TRIBUNALE DI CASSINO,649/23 - 2023,MBCREDIT SOLUTIONS S.P.A.,SERRA GIUSEPPE,ATTO REGISTRATO,no,MBCREDIT


Guardare i nomi in format 'name +1'

In [43]:
df[(df['attore'].str.contains('\+')) | (df['convenuto'].str.contains('\+')) ]

,contratto,debitore,ufficio,atto,emesso da,repertorio,attore,convenuto,stato,compila,Cliente
4403,CRS18966047,SCANZIANI MARIA JOSELLA,MONZA UT DPMB,ATTO VARIO n. 853 del 2021,TRIBUNALE,1064 - 2021,MORELLO + 1,GROSOLI,ATTO REGISTRATO,no,MBCREDIT


Toglie il +1 a chi lo ha

In [44]:
mask_attore = df['attore'].str.contains('\+')
mask_convenuto = df['convenuto'].str.contains('\+')

df.loc[mask_attore, 'attore'] = df.loc[mask_attore, 'attore'].str.split('\+').str[0].str.strip()
df.loc[mask_convenuto, 'convenuto'] = df.loc[mask_convenuto, 'convenuto'].str.split('\+').str[0].str.strip()


In [45]:
# df = pd.DataFrame(results.explode().tolist())
df.to_excel(f'Output/{str(today)}_output_totale_1000_v1.xlsx',index=False)


In [46]:
mask = df.apply(lambda row: (row['Cliente'] in row['attore']) or (row['attore'] in row['Cliente']) or (row['convenuto'] in row['debitore']), axis=1)

df[mask].to_excel(f'Output/{str(today)}_esiti_positivi_1000_v1.xlsx',index=False)